#**SCRIPT PER LA VERIFICA DELLE ANNOTAZONI**

Tramite il seguente script si verifica se l'annotazione è stata effettuata correttamente dagli annotatori. Il formato dell'annotazioni è il seguente

"class_id", "x_center", "y_center", "width height", "x1", "y1", "visibility"

Un'esempio con **1 box** e un **1 keypoint** che rende l'idea:

| class_id | x_center | y_center | width  | height | x_kp | y_kp | visibility|
| ---------| -------- | ---------| -------| -------|------|------|-----------|
| 0        | 0.2577   | 0.2751   | 0.1625 | 0.2088 |0.2715|0.2751| 2         |

Dove:

**v = 2 → visibile e annotato manualmente**

**v = 1 → non visibile ma presente nella struttura (es. occluso)**

**v = 0 → non presente o non annotabile (placeholder)**



---



In [ ]:
# === MOUNT DRIVE ===
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **VERSIONE 2.0 (CON LOG DINAMICO)**

⚠ Utilizzare questa versione di codice ⚠

Questa versione consente di generare un file .csv contenente keypoints per bounded box con il seguente schema:

| file\_name      | num\_kp\_box\_01 | num\_kp\_box\_02 | num\_kp\_box\_03 |
| --------------- | ---------------- | ---------------- | ---------------- |
| staffa\_001.jpg | 2                |                  |                  |
| staffa\_002.jpg | 4                | 3                |                  |
| staffa\_003.jpg | 5                | 5                | 4                |




---



In [ ]:
import os
import cv2
import csv

# === PERCORSI CARTELLE ===
images_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_GoodAnnotated_PCTO_Agostino/images"
labels_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_GoodAnnotated_PCTO_Agostino/labels"
output_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_GoodAnnotated_PCTO_Agostino/output"
csv_log_path = os.path.join(output_dir, "numero_keypoints_log.csv")

# === PARAMETRI VISUALIZZAZIONE ===
bbox_color = (0, 255, 0)
kp_color = (0, 0, 255)
radius = 25
thickness = 20
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 3
text_thickness = 10
text_color = (0, 0, 255)

# === CREA CARTELLA DI OUTPUT ===
os.makedirs(output_dir, exist_ok=True)

# === FILTRA E PRENDI PRIME 5 IMMAGINI ===
file_list = sorted(os.listdir(images_dir))
file_list = [f for f in file_list if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
#file_list = file_list[:5]  # solo le prime 5 per test (COMMENTARLO SE SI VOGLIONO FARE TUTTE)

# === CSV LOG DINAMICO ===
csv_log = []
max_boxes = 0  # si utilizza x contare solo le colonne necessarie

for file_name in file_list:
    image_path = os.path.join(images_dir, file_name)
    label_path = os.path.join(labels_dir, os.path.splitext(file_name)[0] + '.txt')

    if not os.path.exists(label_path):
        print(f"⚠️ Nessuna label trovata per {file_name}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Errore nel leggere l'immagine: {image_path}")
        continue

    h, w = image.shape[:2]
    kp_counts = []  # keypoints per bounding box

    with open(label_path, 'r') as f:
        lines = f.readlines()

    index = 1
    for box_index, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) < 6:
            continue

        class_id = int(parts[0])
        x_center, y_center, bw, bh = map(float, parts[1:5])
        keypoints = list(map(float, parts[5:]))

        # Bounding box
        x1 = int((x_center - bw / 2) * w)
        y1 = int((y_center - bh / 2) * h)
        x2 = int((x_center + bw / 2) * w)
        y2 = int((y_center + bh / 2) * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, thickness)

        # Keypoints per box
        count = 0

        for i in range(0, len(keypoints), 3):
            xk = int(keypoints[i] * w)
            yk = int(keypoints[i + 1] * h)
            v = int(keypoints[i + 2])

            if v > 0:
                cv2.circle(image, (xk, yk), radius, kp_color, -1)
                cv2.putText(image, str(index), (xk + 5, yk - 5), font, font_scale, text_color, text_thickness)
                count += 1
                index += 1

        kp_counts.append(count)

    # Salva immagine annotata
    output_path = os.path.join(output_dir, file_name)
    cv2.imwrite(output_path, image)
    print(f"✅ Annotata e salvata: {file_name}")

    # Salva il conteggio nel log
    max_boxes = max(max_boxes, len(kp_counts))
    csv_log.append((file_name, *kp_counts))

# === CREAZIONE HEADER CSV ===
header = ["file_name"] + [f"num_kp_box_{i+1:02d}" for i in range(max_boxes)]

# === SCRIVI IL CSV ===
with open(csv_log_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(csv_log)

print(f"\n📄 CSV salvato con successo: {csv_log_path}")
print("✅ Tutte le immagini elaborate.")


✅ Annotata e salvata: staffe_001.jpg
✅ Annotata e salvata: staffe_002.jpg
✅ Annotata e salvata: staffe_003.jpg
✅ Annotata e salvata: staffe_004.jpg
✅ Annotata e salvata: staffe_005.jpg
✅ Annotata e salvata: staffe_006.jpg
✅ Annotata e salvata: staffe_007.jpg
✅ Annotata e salvata: staffe_008.jpg
✅ Annotata e salvata: staffe_009.jpg
✅ Annotata e salvata: staffe_010.jpg

📄 CSV salvato con successo: /content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_GoodAnnotated_PCTO_Agostino/output/numero_keypoints_log.csv
✅ Tutte le immagini elaborate.


# **NUOVA VERSIONE CHE FA SORTING PER EVITARE ERRORI**

In [ ]:
import os
import cv2
import csv

# === PERCORSI CARTELLE ===
images_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/images"
labels_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/labels"
output_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/output"
csv_log_path = os.path.join(output_dir, "numero_keypoints_log.csv")

# === PARAMETRI VISUALIZZAZIONE ===
bbox_color = (0, 255, 0)
kp_color = (0, 0, 255)
radius = 25
thickness = 20
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 3
text_thickness = 10
text_color = (0, 0, 255)

# === CREA CARTELLA DI OUTPUT ===
os.makedirs(output_dir, exist_ok=True)

# === ORDINA E ABBINA FILE ===
image_files = sorted([f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
label_files = sorted([f for f in os.listdir(labels_dir) if f.lower().endswith('.txt')])

image_names = [os.path.splitext(f)[0] for f in image_files]
label_names = [os.path.splitext(f)[0] for f in label_files]

# Tieni solo i file che hanno sia immagine che label
matched_files = sorted(set(image_names) & set(label_names))
print(f"🔍 Trovati {len(matched_files)} file validi con immagine e label.")

# === CSV LOG DINAMICO ===
csv_log = []
max_boxes = 0  # per contare il massimo numero di colonne nel CSV

for base_name in matched_files:
    image_ext = next((ext for ext in ['.jpg', '.jpeg', '.png'] if os.path.exists(os.path.join(images_dir, base_name + ext))), None)
    if not image_ext:
        print(f"❌ Estensione immagine non trovata per {base_name}")
        continue

    image_path = os.path.join(images_dir, base_name + image_ext)
    label_path = os.path.join(labels_dir, base_name + '.txt')

    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ Errore nel leggere l'immagine: {image_path}")
        continue

    h, w = image.shape[:2]
    kp_counts = []

    with open(label_path, 'r') as f:
        lines = f.readlines()



    for box_index, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) < 6:
            continue

        class_id = int(parts[0])
        x_center, y_center, bw, bh = map(float, parts[1:5])
        keypoints = list(map(float, parts[5:]))

        # Bounding box
        x1 = int((x_center - bw / 2) * w)
        y1 = int((y_center - bh / 2) * h)
        x2 = int((x_center + bw / 2) * w)
        y2 = int((y_center + bh / 2) * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, thickness)

        # Keypoints
        count = 0
        index = 1
        for i in range(0, len(keypoints), 3):
            xk = int(keypoints[i] * w)
            yk = int(keypoints[i + 1] * h)
            v = int(keypoints[i + 2])
            if v > 0:
                cv2.circle(image, (xk, yk), radius, kp_color, -1)
                cv2.putText(image, str(index), (xk + 5, yk - 5), font, font_scale, text_color, text_thickness)
                count += 1
                index += 1

        kp_counts.append(count)

    output_path = os.path.join(output_dir, base_name + image_ext)
    cv2.imwrite(output_path, image)
    print(f"✅ Annotata e salvata: {base_name + image_ext}")

    max_boxes = max(max_boxes, len(kp_counts))
    csv_log.append((base_name + image_ext, *kp_counts))

# === HEADER CSV ===
header = ["file_name"] + [f"num_kp_box_{i+1:02d}" for i in range(max_boxes)]

# === SALVA CSV ===
with open(csv_log_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(csv_log)

print(f"\n📄 CSV salvato con successo: {csv_log_path}")
print("✅ Tutte le immagini elaborate.")


🔍 Trovati 257 file validi con immagine e label.
✅ Annotata e salvata: staffe_001.jpg
✅ Annotata e salvata: staffe_002.jpg
✅ Annotata e salvata: staffe_003.jpg
✅ Annotata e salvata: staffe_004.jpg
✅ Annotata e salvata: staffe_005.jpg
✅ Annotata e salvata: staffe_006.jpg
✅ Annotata e salvata: staffe_007.jpg
✅ Annotata e salvata: staffe_008.jpg
✅ Annotata e salvata: staffe_009.jpg
✅ Annotata e salvata: staffe_010.jpg
✅ Annotata e salvata: staffe_011.jpg
✅ Annotata e salvata: staffe_012.jpg
✅ Annotata e salvata: staffe_013.jpg
✅ Annotata e salvata: staffe_014.jpg
✅ Annotata e salvata: staffe_015.jpg
✅ Annotata e salvata: staffe_016.jpg
✅ Annotata e salvata: staffe_017.jpg
✅ Annotata e salvata: staffe_018.jpg
✅ Annotata e salvata: staffe_019.jpg
✅ Annotata e salvata: staffe_020.jpg
✅ Annotata e salvata: staffe_021.jpg
✅ Annotata e salvata: staffe_022.jpg
✅ Annotata e salvata: staffe_023.jpg
✅ Annotata e salvata: staffe_024.jpg
✅ Annotata e salvata: staffe_025.jpg
✅ Annotata e salvata: staff

# **TROVO NEL FILE .CSV IL NUMERO MASSIMO DI KEYPOINT**
Perchè farlo? perchè così capisco quanti keypoints con flag "invisibile" mettere per ciascuna immagine annotata.

In [ ]:
import csv

# === PERCORSO AL FILE CSV ===
csv_path = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/output/numero_keypoints_log.csv"

max_kp = 0

with open(csv_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        for key, value in row.items():
            if key and key.startswith("num_kp_box") and value is not None and value.strip():
                try:
                    val = int(value)
                    if val > max_kp:
                        max_kp = val
                except ValueError:
                    continue  # in caso ci siano celle vuote o non numeriche

print(f" Numero max. di keypoints in una bounding box: {max_kp}")


 Numero max. di keypoints in una bounding box: 11


# **SISTEMAZIONE ANNOTAZIONE**

1) Analizza ogni bounding box (riga del file)

2) Conta i keypoints presenti (basato su triple x y v)

3) Se < 14 keypoints → aggiunge fittizi 0.0 0.0 0 fino a **15**

4) Salva i file corretti in una nuova cartella di output



---



In [ ]:
import os

input_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/labels"
output_dir = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_Nostri/Dataset_Clean_GoodAnnotation/labels_corrette"
num_kp_target = 7  # numero di keypoints da raggiungere (fissi)

# === CREA CARTELLA DI OUTPUT ===
os.makedirs(output_dir, exist_ok=True)

# === PROCESSA OGNI FILE .TXT ===
for file_name in sorted(os.listdir(input_dir)):
    if not file_name.endswith('.txt'):
        continue

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    with open(input_path, 'r') as f:
        lines = f.readlines()

    fixed_lines = []

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue  # salta righe malformate!!!!!!!!!!

        # Estrai le parti
        class_id = parts[0]
        bbox = parts[1:5]
        keypoints = parts[5:]

        # Conta i keypoints reali (ogni keypoint ha 3 valori)
        kp_current = len(keypoints) // 3
        kp_missing = num_kp_target - kp_current

        # Aggiungi keypoints fittizi se mancano
        if kp_missing > 0:
            keypoints += ['0.0', '0.0', '0'] * kp_missing

        # Ricostruisci la riga
        fixed_line = " ".join([class_id] + bbox + keypoints)
        fixed_lines.append(fixed_line)

    # Scrivi il nuovo file
    with open(output_path, 'w') as f:
        for line in fixed_lines:
            f.write(line + "\n")

    print(f"✅ Fix completato: {file_name}")

print("\Tutti i file sono stati sistemati in:", output_dir)


✅ Fix completato: staffe_001.txt
✅ Fix completato: staffe_002.txt
✅ Fix completato: staffe_003.txt
✅ Fix completato: staffe_004.txt
✅ Fix completato: staffe_005.txt
✅ Fix completato: staffe_006.txt
✅ Fix completato: staffe_007.txt
✅ Fix completato: staffe_008.txt
✅ Fix completato: staffe_009.txt
✅ Fix completato: staffe_010.txt
✅ Fix completato: staffe_011.txt
✅ Fix completato: staffe_012.txt
✅ Fix completato: staffe_013.txt
✅ Fix completato: staffe_014.txt
✅ Fix completato: staffe_015.txt
✅ Fix completato: staffe_016.txt
✅ Fix completato: staffe_017.txt
✅ Fix completato: staffe_018.txt
✅ Fix completato: staffe_019.txt
✅ Fix completato: staffe_020.txt
✅ Fix completato: staffe_021.txt
✅ Fix completato: staffe_022.txt
✅ Fix completato: staffe_025.txt
✅ Fix completato: staffe_026.txt
✅ Fix completato: staffe_027.txt
✅ Fix completato: staffe_028.txt
✅ Fix completato: staffe_029.txt
✅ Fix completato: staffe_030.txt
✅ Fix completato: staffe_031.txt
✅ Fix completato: staffe_037.txt
✅ Fix comp